In [69]:
# data import and cleaning
import numpy as np

with open(f"../data/data.txt", 'r', encoding='utf-8-sig') as f:
    file_list = f.readlines()

    pointer = 0
    for i, line in enumerate(file_list):
        file_list[i] = [float(item) for item in line.replace(' ', '').split(',')]

    # first line is number of warehouses
    num_warehouses = int(file_list[pointer][0])
    pointer += 1

    # second line is number of goods
    num_goods = int(file_list[pointer][0])
    pointer += 1

    # third line is number of retailers
    num_retailers = int(file_list[pointer][0])
    pointer += 1

    # fourth line is the capacity of each warehouse
    warehouse_capacities = [int(value) for value in file_list[pointer]]
    assert len(warehouse_capacities) == num_warehouses
    pointer += 1

    # next block is the demand of each retailer for each good and the shortage unit cost
    retailer_good_demand = np.ndarray(shape=(num_retailers, num_goods))
    retailer_good_shortage_cost = np.ndarray(shape=(num_retailers, num_goods))
    for line in file_list[pointer:pointer + num_retailers * num_goods]:
        retailer = int(line[0]) - 1
        good = int(line[1]) - 1
        demand = int(line[2])
        shortage_cost = line[3]
        retailer_good_demand[retailer, good] = demand
        retailer_good_shortage_cost[retailer, good] = shortage_cost

    pointer += num_retailers * num_goods

    # next line is the sizes of each of the goods
    good_sizes = file_list[pointer]
    assert len(good_sizes) == num_goods
    pointer += 1

    # next line is the holding cost of each good
    good_holding_costs = file_list[pointer]
    assert len(good_holding_costs) == num_goods
    pointer += 1

    # next block is the initial stock of each good at each warehouse
    # we initialise as zeroes in case the file doesn't include zero stock goods
    warehouse_good_initial_stock = np.zeros(shape=(num_warehouses, num_goods))
    for line in file_list[pointer:pointer + num_warehouses * num_goods]:
        warehouse = int(line[0]) - 1
        good = int(line[1]) - 1
        initial_stock = int(line[2])
        warehouse_good_initial_stock[warehouse, good] = initial_stock

    pointer += num_warehouses * num_goods

    # the last block is the unit transportation cost for each good between each retailer and each warehouse
    # we initialise as zeroes in case the file doesn't include goods with zero transportation costs
    transportation_costs = np.zeros(shape=(num_retailers, num_warehouses, num_goods))
    for line in file_list[pointer:pointer + num_warehouses * num_retailers * num_goods]:
        warehouse = int(line[0]) - 1
        retailer = int(line[1]) - 1
        good = int(line[2]) - 1
        transport_cost = line[3]
        transportation_costs[warehouse, retailer, good] = transport_cost

print(f"""Data loaded successfully!
    - num_warehouses: {num_warehouses}
    - num_goods: {num_goods}
    - num_retailers {num_retailers}
    - warehouse_capacities: {warehouse_capacities}
    - retailer_good_demand:\n {retailer_good_demand}
    - retailer_good_shortage_cost:\n {retailer_good_shortage_cost}
    - good_sizes: {good_sizes}
    - good_holding_costs: {good_holding_costs}
    - warehouse_good_initial_stock:\n {warehouse_good_initial_stock}
    - transportation_costs:\n {transportation_costs}
""")

Data loaded successfully!
    - num_warehouses: 5
    - num_goods: 3
    - num_retailers 5
    - warehouse_capacities: [800, 1000, 700, 1000, 600]
    - retailer_good_demand:
 [[200. 150. 350.]
 [250. 200. 400.]
 [150. 300. 200.]
 [300. 150. 150.]
 [200. 100. 200.]]
    - retailer_good_shortage_cost:
 [[2.5 3.  2. ]
 [3.  2.  3. ]
 [2.  1.5 1.5]
 [1.5 3.  3. ]
 [2.5 2.5 2.5]]
    - good_sizes: [1.0, 0.8, 0.6]
    - good_holding_costs: [2.0, 5.0, 3.0]
    - warehouse_good_initial_stock:
 [[300. 200. 300.]
 [400. 300. 300.]
 [200. 100. 200.]
 [ 50. 150. 300.]
 [150. 150. 200.]]
    - transportation_costs:
 [[[0.6 0.8 1. ]
  [0.7 0.9 1. ]
  [1.3 1.5 0.8]
  [1.4 1.3 2. ]
  [1.8 1.6 1.2]]

 [[2.2 1.3 1.6]
  [1.9 2.1 2.5]
  [1.  2.5 1.5]
  [1.8 1.7 1.4]
  [1.2 1.9 2.3]]

 [[2.5 0.7 0.8]
  [0.4 1.3 1.4]
  [0.9 1.7 2.3]
  [1.8 2.3 1.8]
  [0.9 1.4 2.4]]

 [[1.6 1.9 0.6]
  [1.4 2.1 2. ]
  [0.6 1.  1.7]
  [1.5 2.2 2.6]
  [1.  1.6 1.9]]

 [[1.8 0.7 0.5]
  [1.8 1.4 2.1]
  [2.5 1.8 1.7]
  [2.6 1.8 2

In [72]:
import gurobipy as gp

m = gp.Model()

warehouses = range(num_warehouses)
goods = range(num_goods)
retailers = range(num_retailers)

warehouse_stock = m.addMVar((num_warehouses, num_goods), lb=0, ub=10000, vtype='I')
transport = m.addMVar((num_warehouses, num_retailers, num_goods), lb=0, ub=10000, vtype='I')
retailer_shortage_cost = m.addMVar(num_retailers, lb=0, ub=10000)
retailer_carry_cost = m.addMVar(num_retailers, lb=0, ub=10000)
warehouse_carry_cost = m.addMVar(num_retailers, lb=0, ub=10000)


m.setObjective(
    gp.quicksum(transport[w, r, g] * transportation_costs[w, r, g] for w in warehouses for r in retailers for g in goods) +
    gp.quicksum(retailer_shortage_cost[r] for r in retailers) +
    gp.quicksum(retailer_carry_cost[r] for r in retailers) +
    gp.quicksum(warehouse_carry_cost[w] for w in warehouses),
    gp.GRB.MINIMIZE
)

# total amount goods at each warehouse cannot exceed capacity
m.addConstrs((gp.quicksum(warehouse_stock[w, g] * good_sizes[g] for g in goods) <= warehouse_capacities[w] for w in warehouses), name="WarehouseCapacities")

# total amount of good sent from warehouse must not exceed stock
m.addConstrs((gp.quicksum(transport[w, r, g] for r in retailers) <= warehouse_stock[w, g] for w in warehouses for g in goods), name="TransportCannotExceedStock")

# carry cost for each retailer is equal to the number of extra goods sent times their unit carry cost
m.addConstrs((retailer_carry_cost[r] >= gp.quicksum((gp.quicksum(transport[w, r, g] for w in warehouses) - retailer_good_demand[r, g]) * good_holding_costs[g] for g in goods) for r in retailers), name="RetailerCarryCost")

# carry cost for each warehouse is equal to the number of remaining goods at the warehouse times their unit carry cost
m.addConstrs(
    (
        warehouse_carry_cost[w] >= gp.quicksum((warehouse_stock[w, g] - gp.quicksum(transport[w, r, g] for r in retailers)) * good_holding_costs[g] for g in goods) for w in warehouses
    ), name="WarehouseCarryCost"
)

# shortage cost for each retailer is equal to the number of units of unmet demand times their shortage costs
m.addConstrs(
    (
        retailer_shortage_cost[r] >= gp.quicksum((retailer_good_demand[r, g] - gp.quicksum(transport[w, r, g] for w in warehouses)) * retailer_good_shortage_cost[r, g] for g in goods) for r in retailers
    ), name="RetailerShortageCost"
)

# # total amount of each good sent to each retailer must equal retailer's demand
# m.addConstrs((gp.quicksum(transport[w, r, g] for w in warehouses) <= retailer_good_demand[r, g] for r in retailers for g in goods), name="RetailerGoodDemand")

m.optimize()
# modelling stuff goes here

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 9 7950X 16-Core Processor, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 16 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 35 rows, 105 columns and 360 nonzeros
Model fingerprint: 0x065586ad
Variable types: 15 continuous, 90 integer (0 binary)
Coefficient statistics:
  Matrix range     [6e-01, 5e+00]
  Objective range  [4e-01, 3e+00]
  Bounds range     [1e+04, 1e+04]
  RHS range        [6e+02, 3e+03]
Found heuristic solution: objective 7650.0000000
Presolve removed 16 rows and 18 columns
Presolve time: 0.00s
Presolved: 19 rows, 87 columns, 237 nonzeros
Variable types: 0 continuous, 87 integer (0 binary)

Root relaxation: objective 2.155064e+03, 12 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time



In [71]:
print(f"""
Results:
    - retailer_shortage_cost: {[float(item.x) for item in retailer_shortage_cost]}
    - retailer_carry_cost: {[float(item.x) for item in retailer_carry_cost]}
    - warehouse_carry_cost: {[float(item.x) for item in warehouse_carry_cost]}
""")


Results:
    - retailer_shortage_cost: [271.0, 1.0, 1050.0, 450.0, 395.0]
    - retailer_carry_cost: [0.0, 0.0, 0.0, 0.0, 0.0]
    - warehouse_carry_cost: [0.0, 0.0, 0.0, 0.0, 0.0]

